# Convolutional Neural Network

#### Processing the MNIST Dataset: Classification (Digits 0 and 9)


In [ ]:
import numpy as np
from scipy import signal
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [1]:
import numpy as np
from scipy import signal
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
def preprocess_data(x, y, limit):
    # Reshape and normalize input data
    x = x.reshape(x.shape[0], 1, 28, 28)
    x = x.astype("float32") / 255
    # Encode output which is a number in range [0,9] into a vector of size 10
    y = to_categorical(y)  
    y = y.reshape(y.shape[0], 10, 1)
    return x[:limit], y[:limit]

#### Load Mnist dataset as is from keras


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape


((60000, 28, 28), (60000,))

In [4]:
print(f"Pixel values of first x_train image: \n{x_train[0]}")

Pixel values of first x_train image: 
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0 

In [5]:
print(f"True category of first example: {y_train[0]}")

True category of first example: 5


#### Now process the data

In [6]:
# load MNIST from server, limit to 100 images per class since we're not training on GPU
x_train, y_train = preprocess_data(x_train, y_train, 1000)
x_test, y_test = preprocess_data(x_test, y_test, 100)

In [8]:
x_train.shape, y_train.shape

((1000, 1, 28, 28), (1000, 10, 1))

In [9]:
print(f"pixel values of first x_train image: \n{x_train[0]}")

pixel values of first x_train image: 
[[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         

In [10]:
print(f"one-hot encoded array of first y_train image: \n{y_train[0]}")

one-hot encoded array of first y_train image: 
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]


### Now I need a function that performs prediction on my neural network

In [11]:
# Performs prediction 
def predict(network, input, training):
    output = input
    for layer in network:
        output = layer.forward(output, training)
    return output

### I also need a function that will perform forward propogation, calculate loss, backpropogate and update weights to minimize loss

In [12]:
def train(network, loss, loss_prime, x_train, y_train, epochs, learning_rate, verbose=True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train): # Will iterate over each example
            # Forward pass on neural network
            output = predict(network, x, True)

            # Accumulate error
            error += loss(y, output)

            # Backward pass on neural network
            grad = loss_prime(y, output) # Derivate of loss with respect to prediction of neural network output
            for layer in reversed(network): # Start from last layer to perform back propogation
                grad = layer.backward(grad, learning_rate) # calculating gradient of each layer and stepping when needed in each layer

        # Calculate average error over training set and print per epoch
        error /= len(x_train) 
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")

### Designing a Base Layer for Neural Networks

Before writing the code for any layers, I will start off with a base layer that will define the common structure and interface (forward and backward methods) for all layers. This will ensure consistency, modularity, and reusability across different types of layers in the neural network.


In [14]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward(self, input, training):
        pass
    
    def backward(self, output_gradient, learning_rate):
        pass

## **The main star of the show: The Convolutional Layer**


In [15]:
class Convolutional(Layer):
    # Input shape = depth x height x width
    # kernel_size = size of each matrix of each kernel
    # depth = number of kernels
    # input depth is different than depth
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        # In this convolutional layer the formula to calculate output shape is [depth, (input size - kernel size) + 1] 
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1) 
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size) # (Number of kernels X input depth X kernel size X kernel size)
        self.kernels = np.random.randn(*self.kernels_shape) # 4d matrix of the weights
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input, training):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth): # Iterate up to number of kernel times
            for j in range(self.input_depth): # Iterate up to input depth times
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid") # Calculate valid cross correlation between single input image and kernel
        return self.output # This will have number of kernels amount of output matrices
    
    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                pass

## Lets break down the math forward pass first
``` 
def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth): 
            for j in range(self.input_depth): 
                self.output[i] += signal.correlate2d(self.input[j], self.kernel[i, j], "valid") 
        return self.output 
``` 


### The Math Behind Valid Cross-Correlation with Matrices

This function operates on a given matrix: $I_{n \times n}$. The matrix $I$ also includes a color channel dimension, but since the channel size is 1, we simplify it as $n \times n$ for simplicity.

Additionally, there is another matrix called the **kernel**, denoted as $K_{m \times m}$. The operation performed between these two matrices is called **valid cross-correlation**, represented as $I \star K$.

#### Formula for Valid Cross-Correlation
For a matrix $A_{n \times n}$ (input) and a matrix $B_{m \times m}$ (kernel), the valid cross-correlation at position $(i, j)$ is defined as:

$
(A \star B)[i, j] = \sum_{p=0}^{m-1} \sum_{q=0}^{m-1} A[i + p, j + q] \cdot B[p, q]
$

Here:
- $A$: Input matrix
- $B$: Kernel matrix
- $i, j$: Indices of the **output matrix**

#### Output Matrix Dimensions
Let $O$ be the output matrix such that $O$ = $(A \star B)[i, j]$. Then the size of O is calculated as:
$
\text{Output size} = (\text{Input size} - \text{Kernel size}) + 1
$

For example:
- Input size: $4 \times 4$
- Kernel size: $2 \times 2$

$
\text{Output size} = (4 - 2) + 1 = 3 \times 3
$

#### Example Calculation
Let the input matrix $A$, kernel $B$, and output matrix $O$ be defined as:  
$
A = \begin{bmatrix} 
1 & 2 & 3 & 4 \\ 
5 & 6 & 7 & 8 \\ 
9 & 10 & 11 & 12 \\ 
13 & 14 & 15 & 16 
\end{bmatrix}, \quad
B = \begin{bmatrix} 
1 & -2 \\ 
2 & 0 
\end{bmatrix}, \quad
O = \begin{bmatrix} 
a_{00} & a_{01} & a_{02} \\ 
a_{10} & a_{11} & a_{12} \\ 
a_{20} & a_{21} & a_{22} 
\end{bmatrix}
$

To compute $O[0, 0]$ (the output at position $i = 0, j = 0$):
$
O[0, 0] = \sum_{p=0}^{1} \sum_{q=0}^{1} A[0 + p, 0 + q] \cdot B[p, q]
$

Step-by-step:
1. $A_{[0, 0]} \cdot B_{[0, 0]} = 1 \cdot 1 = 1$
2. $A_{[0, 1]} \cdot B_{[0, 1]} = 2 \cdot (-2) = -4$
3. $A_{[1, 0]} \cdot B_{[1, 0]} = 5 \cdot 2 = 10$
4. $A_{[1, 1]} \cdot B_{[1, 1]} = 6 \cdot 0 = 0$

Sum these results:
$
O[0, 0] = 1 + (-4) + 10 + 0 = 7
$

#### Final Step
Repeat this computation for all $i, j$ in the output matrix $O$.

The following line of code does exactly this computation but iterates over the depth of the input(amount of color channels) $ j $ and the depth of the kernel (amount of kernels) $ i $.
```
self.output[i] += signal.correlate2d(self.input[j], self.kernel[i, j], "valid")
```


## Here is the second star of the show: Backward Propogation on the Convolutional Layer

The main question is: **how does the loss change when I modify the parameters in the kernel?** To answer this, we must determine how each weight in each kernel impacts the loss. This requires computing the **gradient of the loss with respect to each weight in the kernel**.

### Simplifying the Gradient Calculation

The gradient of the kernel $K_{[i, j]}$ at position $(i, j)$ can be simplified by combining:
1. The **gradient of the loss with respect to the output of the current layer**.
2. The **gradient of the output with respect to the kernel weights**.

In general:
$$
\text{Gradient of Kernel} = \text{Output Gradient} \cdot \text{Input to the Layer}.
$$

### Forward Pass Formula

To understand this better, remember the formula for the forward pass of a convolution operation. For the output of a convolutional layer $Y_{[i, j]}$, we compute:
$$
Y_{[i, j]} = (A \star B)[i, j] = \sum_{p=0}^{m-1} \sum_{q=0}^{m-1} A_{[i + p, j + q]} \cdot B_{[p, q]}
$$
This expands as:
$$
Y_{[i, j]} = A_{[i + 0, j + 0]} \cdot B_{[0, 0]} + A_{[i + 0, j + 1]} \cdot B_{[0, 1]} + \dots + A_{[i + m - 1, j + m - 1]} \cdot B_{[m - 1, m - 1]}.
$$

Here:
- $A$: The input to the convolutional layer.
- $B$: The kernel.
- $\star$: The convolution operator.

### Calculating the Gradient

To compute the gradient $\frac{\partial Y_{[i, j]}}{\partial B_{[p, q]}}$, consider the contribution of each kernel weight $B_{[p, q]}$ to $Y_{[i, j]}$. The derivative is:
$$
\frac{\partial Y_{[i, j]}}{\partial B_{[p, q]}} = A_{[i + p, j + q]}.
$$
This result shows that changing $B_{[p, q]}$ only affects $Y_{[i, j]}$ through the corresponding input value $A_{[i + p, j + q]}$.

This derivative is referred to as the **local gradient**, as it demonstrates the relationship between the kernel and the output.

### Incorporating the Chain Rule

To compute the full gradient of the loss with respect to $B_{[p, q]}$, we must multiply the local gradient by the gradient of the loss from the next layer:
$$
\frac{\partial L}{\partial B_{[p, q]}} = \frac{\partial L}{\partial Y_{[i, j]}} \cdot \frac{\partial Y_{[i, j]}}{\partial B_{[p, q]}}.
$$

Expanding this, we have:
$$
\frac{\partial L}{\partial B_{[p, q]}} = \frac{\partial L_{[i, j]}}{\partial Y_{[i, j]}} \cdot A_{[i + p, j + q]}.
$$

This gives the full gradient of the kernel, which is then used to update the kernel weights during backpropagation.


In [16]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input, training):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))
    
    def print(self, input):
        print(f"Activation {self.activation(input)}")
        print(f"Activation prime {self.activation_prime(input)}")


In [17]:
class Relu(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0 ,x)
        
        def relu_prime(x):
            return (x > 0).astype(float)

        super().__init__(relu, relu_prime)


### Using ReLU Activation with He Initialization

This network uses the **ReLU activation function**. However, I encountered an issue where:  
The *loss decreased very slowly*, and the *predictions were poor*.

To address this, I implemented **He initialization** in my Dense layer. This initialization helps maintain stable activation variances across layers when using ReLU. Specifically, I used the scaling factor:

$
\sqrt{\frac{2}{\text{input\_size}}}
$

This factor scales the initial weights appropriately for layers with ReLU activations, ensuring better convergence and performance.


In [18]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        # The scaling factor "np.sqrt(2 / input_size)" accounts for number of incoming connections to each neuron
        # maintaining stability
        self.weights = np.random.randn(output_size, input_size) *  np.sqrt(2 / input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input, training):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient # Subtracts a small step of learning rate * gradients of weights in order to minimize loss
        self.bias -= learning_rate * output_gradient
        return input_gradient
    
    def print(self):
        print(self.weights.shape)
        print(self.bias.shape)


### Categorical Cross-Entropy Loss Function

Since I am predicting across **10 classes**, I chose the **categorical cross-entropy loss function**. This loss is defined as:

$
L = -\sum_{i=1}^{N} \sum_{j=1}^{C} y_{i,j} \log(\hat{y}_{i,j})
$

where:

- ( N ) is the number of samples,
- ( C ) is the number of classes,
- $( y_{i,j} ) $ is the ground truth for the ( i )-th sample and ( j )-th class, and 
- $( \hat{y}_{i,j} ) $ is the predicted probability for the ( i )-th sample and ( j )-th class.

#### Derivative of the Loss Function
The derivative with respect to the predicted probability $ ( \hat{y}_{i,j} ) $ is:

$
\frac{\partial L}{\partial \hat{y}_{i,j}} = \frac{\hat{y}_{i,j} - y_{i,j}}{N}
$

In [19]:
def categorical_cross_entropy(y_true, y_pred):
  
    return -np.mean(np.sum(y_true * np.log(np.clip(y_pred, 1e-12, 1 - 1e-12)), axis=1))

def categorical_cross_entropy_prime(y_true, y_pred):
    
    return (y_pred - y_true) / np.size(y_true)

The following layers are straightforward: a **reshape layer**, followed by a **Dense** layer.


In [20]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input, training):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)


class Dense(Layer):
    def __init__(self, input_size, output_size):
        # The scaling factor "np.sqrt(2 / input_size)" accounts for number of incoming connections to each neuron
        # maintaining stability
        self.weights = np.random.randn(output_size, input_size) *  np.sqrt(2 / input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input, training):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient # Subtracts a small step of learning rate * gradients of weights in order to minimize loss
        self.bias -= learning_rate * output_gradient
        return input_gradient
    
    def print(self):
        print(self.weights.shape)
        print(self.bias.shape)


And finally, a **softmax** and **dropout** layer is used to output the probability of each class.


In [21]:
class Softmax(Layer):
    def forward(self, input, training):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        # This version is faster than the one presented in the video
        n = np.size(self.output)
        return np.dot((np.identity(n) - self.output.T) * self.output, output_gradient)
        # Original formula:
        # tmp = np.tile(self.output, n)
        # return np.dot(tmp * (np.identity(n) - np.transpose(tmp)), output_gradient)

class Dropout(Layer):
    def __init__(self, rate):
        self.rate = rate
        self.mask = None

    def forward(self, input, training):
        if training:
            self.mask = np.random.binomial(1, 1 - self.rate, size=input.shape) 
            return input * self.mask / (1 - self.rate)
        else:
            return input

    
    def backward(self, output_gradient, learning_rate):
        return output_gradient * self.mask



### Configuring and Training the Neural Network


In [22]:
# Neural network
network = [
    Convolutional((1, 28, 28), 3, 5),
    Relu(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Relu(),
    Dropout(0.05),
    Dense(100, 10),  
    Softmax() 
]

# Train
train(
    network,
    categorical_cross_entropy,
    categorical_cross_entropy_prime,
    x_train,
    y_train,
    epochs=100,
    learning_rate= 0.1,
    verbose=True
)



1/100, error=0.1904140412393742
2/100, error=0.08443746168326244
3/100, error=0.06469184583177913
4/100, error=0.05692088667162284
5/100, error=0.04870857853292249
6/100, error=0.042692483714764404
7/100, error=0.04305004994587654
8/100, error=0.039768175571013566
9/100, error=0.04005235439076175
10/100, error=0.03696713319602728
11/100, error=0.03374244937117737
12/100, error=0.03211482645141877
13/100, error=0.029327654184351182
14/100, error=0.02909696491791252
15/100, error=0.02958812083174631
16/100, error=0.02851921739323268
17/100, error=0.023348829001051563
18/100, error=0.026678822714901677
19/100, error=0.02726098055756334
20/100, error=0.025217623698102487
21/100, error=0.024671114456522546
22/100, error=0.025517934741243833
23/100, error=0.025283755929077258
24/100, error=0.02387131712158869
25/100, error=0.022281336777385148
26/100, error=0.023600828309355937
27/100, error=0.02161376518016846
28/100, error=0.020760396334214382
29/100, error=0.021438469085464498
30/100, err

### Testing the Neural Network on the Testing Set


In [23]:
right = 0
# Test
for x, y in zip(x_test, y_test):
    output = predict(network, x, False)
    if np.argmax(output) == np.argmax(y):
        right += 1
    print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

total = (right / len(x_test)) * 100
print(f"Correct: {total}%")


pred: 7, true: 7
pred: 2, true: 2
pred: 1, true: 1
pred: 0, true: 0
pred: 4, true: 4
pred: 1, true: 1
pred: 4, true: 4
pred: 9, true: 9
pred: 2, true: 5
pred: 9, true: 9
pred: 0, true: 0
pred: 6, true: 6
pred: 9, true: 9
pred: 0, true: 0
pred: 1, true: 1
pred: 5, true: 5
pred: 9, true: 9
pred: 7, true: 7
pred: 3, true: 3
pred: 4, true: 4
pred: 9, true: 9
pred: 6, true: 6
pred: 6, true: 6
pred: 5, true: 5
pred: 4, true: 4
pred: 0, true: 0
pred: 7, true: 7
pred: 4, true: 4
pred: 0, true: 0
pred: 1, true: 1
pred: 3, true: 3
pred: 1, true: 1
pred: 3, true: 3
pred: 6, true: 4
pred: 7, true: 7
pred: 2, true: 2
pred: 7, true: 7
pred: 1, true: 1
pred: 5, true: 2
pred: 1, true: 1
pred: 1, true: 1
pred: 7, true: 7
pred: 9, true: 4
pred: 2, true: 2
pred: 3, true: 3
pred: 5, true: 5
pred: 1, true: 1
pred: 2, true: 2
pred: 4, true: 4
pred: 4, true: 4
pred: 6, true: 6
pred: 3, true: 3
pred: 5, true: 5
pred: 5, true: 5
pred: 6, true: 6
pred: 0, true: 0
pred: 4, true: 4
pred: 1, true: 1
pred: 9, true: